# Lab 4: Model Monitoring, Drift Detection, and Retraining Strategy

Welcome to the final lab! We've built a pipeline, trained a model, and deployed it as an API. But the work isn't done. Models in production can become stale as the world changes. In this lab, we'll learn how to monitor our model for **drift** and outline a strategy for automated retraining to keep our model fresh and accurate.

## Learning Objectives

By the end of this lab, you will be able to:
- Understand the concepts of Data Drift and Concept Drift.
- Use the **Evidently AI** library to generate a drift report.
- Compare a reference dataset (training data) with a current dataset (production data).
- Interpret the drift report to identify changes in your data.
- Formulate a strategy for an automated retraining pipeline.

### 1. Setup: Installing Dependencies

Our key tool for this lab is `evidently`, an open-source library for evaluating, testing, and monitoring ML models.

In [ ]:
%pip install evidently pandas scikit-learn

### 2. Key Concepts: Model Drift

Model drift (or model decay) is the degradation of a model's predictive power due to changes in the environment. There are two main types:

**Data Drift:** The statistical properties of the input features change. For example, the average `Balance` of new customers might increase, or the `Age` distribution might shift. The model is now seeing data it wasn't trained on, which can lead to inaccurate predictions.

**Concept Drift:** The relationship between the features and the target variable changes. For example, a new banking regulation might change what factors cause a customer to churn. The patterns the model learned are no longer valid.

### 3. Simulating Production Data

To detect drift, we need to compare the data our model was trained on (**reference** data) with the data it's seeing now in production (**current** data). Since we don't have a live application, we'll simulate this by taking our original dataset and modifying it to create a 'current' dataset with intentional drift.

In [ ]:
import pandas as pd

# Load the original processed data as our 'reference' dataset
reference_df = pd.read_parquet('../../data/churn_processed.parquet')

# Create a 'current' dataset by sampling and modifying the reference data
current_df = reference_df.sample(n=5, random_state=42).copy()

# Introduce some data drift
print("Original 'current' data:\n", current_df[['Age', 'Balance', 'IsActiveMember']])
current_df['Age'] = current_df['Age'] + 10  # All customers are now older
current_df['Balance'] = current_df['Balance'] * 1.5 # Balances have increased
current_df['IsActiveMember'] = 0 # All of these customers became inactive
print("\nModified 'current' data with drift:\n", current_df[['Age', 'Balance', 'IsActiveMember']])


### 4. Generating a Drift Report with Evidently AI

Now, let's use Evidently to compare our `reference_df` and `current_df`.

In [ ]:
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset

# Create a report and add the DataDriftPreset
data_drift_report = Report(metrics=[
    DataDriftPreset(),
])

# Run the report
data_drift_report.run(reference_data=reference_df, current_data=current_df)

# Display the report directly in the notebook
data_drift_report

### 5. Interpreting the Report

The interactive report above gives you a complete picture of the data drift.

- **Data Drift Detected:** At the top, Evidently gives a clear verdict on whether significant drift has occurred.
- **Feature-level Drift:** You can see a breakdown of each feature. For features where drift is detected (like `Age`, `Balance`, and `IsActiveMember` in our simulation), you can see how their statistical distributions have changed between the reference and current datasets.
- **P-value:** The p-value for each feature's drift test is shown. A low p-value (typically < 0.05) indicates significant drift.

### 6. Strategy for Automated Retraining

Generating a report is great, but in a real MLOps system, we want to act on it. This is where we close the loop by setting up an automated retraining pipeline.

Here's how all the pieces of our tutorial would fit together in a production system:

1. **Scheduled Monitoring:** A scheduled job (e.g., a daily cron job or a Prefect flow) runs the drift detection analysis using Evidently. It compares the latest production data with the original training data.

2. **Drift Threshold:** The job checks the output of the Evidently report (which can be exported as a JSON object). If the number of drifted features or the overall drift score exceeds a predefined threshold, it decides that retraining is necessary.

3. **Trigger Retraining:** If the threshold is breached, the monitoring service makes an API call to trigger a new pipeline run. This could be a webhook to a CI/CD system or, in our case, a call to trigger our **Prefect `etl_flow` from Lab 1**.

4. **Automated ETL and AutoML:** The Prefect flow from Lab 1 runs, processing the new raw data. The output of this flow then becomes the input for our **AutoML process from Lab 2**. A new, better model is trained on the fresh data and registered in the MLflow Registry.

5. **Automated Deployment (Optional):** A CI/CD pipeline could then automatically deploy the new model version, for example, by updating the stage of the new model to "Production", so our FastAPI app from Lab 3 picks it up.

This creates a closed-loop system that automatically detects when a model is going stale and retrains and redeploys a new version, ensuring the business is always using the most accurate model.

### 7. Tutorial Conclusion

**Congratulations!** You have completed the entire MLOps tutorial.

You have gone through the full, end-to-end lifecycle of a machine learning system, mirroring the practices of top AI teams:

- **Lab 1:** You built a scalable ETL pipeline with **Prefect** and **Dask**.
- **Lab 2:** You ran an AutoML workflow with **PyCaret** and tracked it with **MLflow**.
- **Lab 3:** You deployed the model as a resilient **FastAPI** microservice.
- **Lab 4:** You learned how to monitor for drift with **Evidently AI** and designed an automated retraining strategy.

You now have a production-ready MLOps stack running locally and a solid, practical understanding of how to build and maintain real-world machine learning systems.